In [1]:
import tempfile
from abc import ABC, abstractmethod
from copy import deepcopy
from pathlib import Path
from typing import Type

import numpy as np
from ConfigSpace import Configuration, ConfigurationSpace
from smac import AlgorithmConfigurationFacade, Scenario
from smac.runhistory.dataclasses import TrialValue

from src.constant import TEMP_DIR
from src.instance.TSP_Instance import TSP_Instance
from src.main import *
from src.portfolio import Portfolio
from src.solver.TSP_LKH_Solver import TSP_LKH_Solver
import logging


In [2]:
# class PortfolioConstructor(ABC):
#     @abstractmethod
#     def construct(self, size: int, solver_class: Type[TSP_LKH_Solver]) -> Portfolio:
#         pass

In [3]:
K = 2
t_c = 100
training_instances = [
    TSP_Instance("1.tsp", 20887545.00),
    TSP_Instance("2.tsp", 21134211.00),
    TSP_Instance("3.tsp", 21196547.00),
    TSP_Instance("4.tsp", 21428037.00),
    TSP_Instance("5.tsp", 11020488.00),
]
solver_class=TSP_LKH_Solver
portfolio = Portfolio(size=K, solver_class=solver_class)
configuration_time = np.ones(shape=(K,)) * t_c
cost = portfolio.evaluate(training_instances, remaining_time=configuration_time, max_cost=100)

In [ ]:
K = 2
t_c = 100
t_v = 100
training_instances = [
    TSP_Instance("1.tsp", 20887545.00),
    TSP_Instance("2.tsp", 21134211.00),
    TSP_Instance("3.tsp", 21196547.00),
    TSP_Instance("4.tsp", 21428037.00),
    TSP_Instance("5.tsp", 11020488.00),
]
solver_class=TSP_LKH_Solver
n = 2

best_portfolio = None
best_cost = np.inf

for _ in range(n):
    portfolio = Portfolio(size=K, solver_class=solver_class)

    smac_configuration_space = ConfigurationSpace()
    for idx, solver in enumerate(portfolio.solvers):
        for k, v in solver.config.config_space.items():
            v = deepcopy(v)
            v.name = f"{idx}__{v.name}"
            smac_configuration_space.add(v)

    temp_dir = tempfile.TemporaryDirectory(dir=TEMP_DIR)
    output_directory = Path(temp_dir.name)

    scenario = Scenario(
        configspace=smac_configuration_space, 
        output_directory=output_directory,
    )

    smac = AlgorithmConfigurationFacade(
        scenario,
        lambda seed: None,
        overwrite=True,
        logging_level=logging.CRITICAL
    )
    configuration_time = np.ones(shape=(K,)) * t_c

    while configuration_time.sum() > 0:
        trial_info = smac.ask()
        portfolio.update_config(trial_info.config)
        cost = portfolio.evaluate(training_instances, remaining_time=configuration_time, max_cost=100)
        trial_value = TrialValue(cost=cost)
        smac.tell(trial_info, trial_value)

    temp_dir.cleanup()

    incumbent = smac.intensifier.get_incumbent()
    if incumbent is not None:
        portfolio.update_config(incumbent)
        validation_time = np.ones(shape=(K,)) * t_v
        cost = portfolio.evaluate(training_instances, remaining_time=validation_time, max_cost=100)

        if cost < best_cost:
            best_cost = cost
            best_portfolio = portfolio
    else:
        print("No incumbent found")